<a href="https://colab.research.google.com/github/Adhul-R/LLM/blob/main/Weather_Forescast_Agent_with_Pydantic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydantic-ai==0.1.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.2/373.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [6]:
import requests

BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "api key"

# custom weather forecast function
def find_weather(city: str) -> dict:
  """This function returns the current weatehr forecast for the given city"""
  units = "metric"
  params = {
      'q': city,
      'appid': API_KEY,
      'units': units
  }
  response = requests.get(BASE_URL, params=params)
  result = response.json()

  return result

In [7]:
op = find_weather("coimbatore")
print(op)

{'coord': {'lon': 76.9667, 'lat': 11}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 24.88, 'feels_like': 25.59, 'temp_min': 24.88, 'temp_max': 24.88, 'pressure': 1007, 'humidity': 83, 'sea_level': 1007, 'grnd_level': 958}, 'visibility': 6000, 'wind': {'speed': 7.2, 'deg': 210}, 'clouds': {'all': 100}, 'dt': 1748598189, 'sys': {'type': 1, 'id': 9206, 'country': 'IN', 'sunrise': 1748564902, 'sunset': 1748610678}, 'timezone': 19800, 'id': 1273865, 'name': 'Coimbatore', 'cod': 200}


In [8]:
import os
import requests
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

In [9]:
os.environ["GROQ_API_KEY"] = "api key"

In [10]:
# Define the output schema
class WeatherForecast(BaseModel):
    location: str
    description: str
    temperature_celsius: float

In [11]:
weather_agent = Agent(
    model="groq:llama-3.1-8b-instant",
    model_settings=ModelSettings(temperature=0.2),
    output_type=str,
    system_prompt=("You are a helpful weather assistant. Use the 'get_weather_forecast' tool to find current weather "
        "conditions for any city. Provide clean and friendly answers.")
)

In [12]:
# weather forecast tool
@weather_agent.tool
def get_weather_forecast(ctx: RunContext, city: str) -> WeatherForecast:
    """
    Returns current weather forecast for a city using OpenWeatherMap API.
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    api_key = "api key"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'
    }

    res = requests.get(url, params=params).json()

    return WeatherForecast(
        location=res["name"],
        description=res["weather"][0]["description"].capitalize(),
        temperature_celsius=res["main"]["temp"]
    )

In [25]:
question = input("🌤️ Ask about the weather: ")
result = weather_agent.run_sync(question)
print("\n📍 Forecast:", result.output)

🌤️ Ask about the weather: Chennai

📍 Forecast: It's currently 28.06°C in Chennai with a misty weather condition.
